# 🎨 SDXL Vintage Illustration Notebook — PRO

Stable Diffusion XL / Turbo / SD 1.5 preconfigured for **vintage ink+watercolor** illustrations (old book style, retro cartoon aesthetic).

### Included
- **Mode Selector** (CPU / GPU basic / GPU optimal via `CONFIG`)
- **Gallery Manager (Flask web UI)** + **Logs** page
- **Text2Img** (default vintage style + custom scene prompt)
- **Img2Img** (photo → vintage illustration)
- **ControlNet (Canny)** for pose/contour consistency
- **Upscale** (x4) for higher resolution
- **Color tone control** (dropdown presets + custom override)
- Saving images + JSON metadata to `/content/outputs` (timestamped)



In [ ]:
## ✅ Tips

- Change **`color_tone`** or **`custom_tone`** any time, then regenerate.
- Use **`seed=None`** for randomization, or set a fixed integer for repeatability.
- For **Img2Img**, tweak **`strength`**:
  - lower = closer to original,
  - higher = stronger style.
- **ControlNet (Canny)** retains silhouette/contours. For different looks, adjust **`canny_low`/`canny_high`**.
- If memory errors occur, run **`free_memory()`** and re-run only the needed loader.


In [ ]:
# @title 🚀 Repo init
from scripts.repo_init import init_repo

REPO_NAME = "sd-colab-gallery"
ORIGIN_URL = f"https://github.com/tekswirl25/{REPO_NAME}.git"
UPSTREAM_URL = ""  # если работаешь с форком, укажи URL апстрима

repo_dir = init_repo(REPO_NAME, ORIGIN_URL, UPSTREAM_URL)



In [ ]:
# @title 🚀 Install gradio
!pip install gradio


In [ ]:
# @title 🌐 Start Gradio server (Logs + Gallery)
from scripts.server_gradio import start_gradio_server

server = start_gradio_server("/content/outputs")

if server and hasattr(server, "share_url") and server.share_url:
    print(f"✅ Gradio server running at: {server.share_url}")
else:
    print("⚠️ Gradio server started, but no public URL detected")


In [ ]:
# @title 📦 3 Install dependencies
import os

if MODE in ["GPU_OPTIMAL", "GPU_BASIC"]:
    # GPU-варианты: нужен torch с CUDA
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

!pip install diffusers==0.29.0 transformers accelerate safetensors xformers

# Общие пакеты для всех режимов
!pip install opencv-python-headless pillow flask nest_asyncio ipywidgets

print(f"✅ Dependencies installed for MODE={MODE}")



In [ ]:
# @title 🌐 Start Flask server
#from scripts.server import start_server
#from scripts.logger import log_info

#start_server(CONFIG["OUTPUT_DIR"], port=8000)

#log_info("✅ Flask server started on http://127.0.0.1:8000")
#log_info("🌐 Logs available at /logs")
#log_info("🌐 Gallery available at /gallery")


In [ ]:
# @title 🔧 1 Config init
import os, sys, torch
sys.path.append(repo_dir)  # 🚀 Repo init

# -----------------------------
# UI-параметры (выборы перед запуском)
ENV_MODE        = "GPU"        #@param ["GPU", "CPU"]
PROGRAM_VERSION = "SDXL"       #@param ["SDXL", "SDXL_TURBO", "SD15"]
MODE            = "GPU_OPTIMAL" #@param ["GPU_OPTIMAL", "GPU_BASIC", "CPU"]
OUTPUT_DIR      = "/content/outputs"
# -----------------------------

# HuggingFace token (Colab Secrets или вручную)
hf_token = None
try:
    from google.colab import userdata
    hf_token = userdata.get("HF_TOKEN")
except Exception:
    pass

from scripts.config import init_config
from scripts.logger import log_info, log_error

CONFIG, VARIANT, DEFAULTS, AUTO_UPSCALE = init_config(
    model_variant=PROGRAM_VERSION,
    output_dir=OUTPUT_DIR,
    hf_token=hf_token,
    mode=MODE,
    env_mode=ENV_MODE  # новый аргумент
)

log_info(f"Torch version: {torch.__version__}")
log_info(f"Device: {CONFIG['DEVICE']} | DType: {CONFIG['DTYPE']} | Mode: {CONFIG['MODE']}")


In [ ]:
#@title 🔍 1.1 Тест ENV_MODE
from scripts.config import init_config

# CPU режим
config_cpu, *_ = init_config(model_variant="SDXL", env_mode="CPU")
print("CPU test:", config_cpu["DEVICE"], config_cpu["DTYPE"])

# GPU режим (при наличии CUDA)
config_gpu, *_ = init_config(model_variant="SDXL", env_mode="GPU")
print("GPU test:", config_gpu["DEVICE"], config_gpu["DTYPE"])


In [ ]:
# @title 📦 2 Server (Flask gallery + logs)
from scripts.gallery_manager import start_gallery
start_gallery(CONFIG["OUTPUT_DIR"], port=8000, in_colab=True)




In [ ]:
# @title 📦 3 Install dependencies (Colab universal)
!pip install --upgrade pip

import os, sys, subprocess

if ENV_MODE == "GPU":
    subprocess.check_call([sys.executable, "-m", "pip", "install",
        "torch", "torchvision", "torchaudio",
        "--index-url", "https://download.pytorch.org/whl/cu121"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "xformers"])
else:  # CPU fallback
    subprocess.check_call([sys.executable, "-m", "pip", "install",
        "torch", "torchvision", "torchaudio",
        "--index-url", "https://download.pytorch.org/whl/cpu"])
    print("⚠️ CPU mode selected: xformers skipped")

# Общий стек
subprocess.check_call([sys.executable, "-m", "pip", "install",
    "diffusers==0.29.0", "transformers", "accelerate", "safetensors",
    "ipywidgets", "opencv-python-headless", "pillow", "flask", "nest_asyncio"])

import torch
print("✅ Dependencies installed. Torch version:", torch.__version__)


In [ ]:
# @title 🔁 4 Imports & utils
from scripts.utils import (
    ts_now, base_name, save_image_and_meta,
    free_memory, list_images, canny_from_image
)
from scripts.logger import log_info, log_error
log_info(f"Utils loaded. OUTPUT_DIR={CONFIG['OUTPUT_DIR']}")



In [ ]:
# @title 🧠 5 Model loaders
from scripts.loaders import (
    get_txt2img_pipe,
    get_img2img_pipe,
    get_controlnet_pipe,
    get_upscale_pipe,
    reset_pipes
)
log_info(f"Loader functions ready for variant: {CONFIG['MODEL_VARIANT']}")




In [ ]:
# @title 🎨 6 Style base & Prompt builder
from scripts.prompt_builder import build_prompt

user_prompt = "A thoughtful man, slightly resembling donald trump."  #@param {type:"string"}
base_style  = "illustration"  #@param ["photoreal","illustration","anime"]
color_tone  = "vintage"       #@param ["warm","cool","vintage"]
negative    = ""              #@param {type:"string"}

final_prompt = build_prompt(user_prompt, style=base_style, tone=color_tone)
log_info(f"Prompt built. style={base_style}, tone={color_tone}")



In [ ]:
# @title 🖼 Text2Img
from PIL import Image
from scripts.config import CONFIG, DEVICE, DTYPE, VARIANT_MODELS, DEFAULTS
from scripts.loaders import get_txt2img_pipe
from scripts.utils import save_image_and_meta, ts_now
from scripts.logger import log_info
import torch



variant = CONFIG["MODEL_VARIANT"]
model_id = VARIANT_MODELS[variant]["txt2img"]

pipe = get_txt2img_pipe(model_id, DEVICE, DTYPE)

steps     = DEFAULTS["txt2img_steps"]
cfg_scale = DEFAULTS["txt2img_cfg"]
height, width = DEFAULTS["img_size"]

seed = 12345      #@param {type:"number"}
n    = 1          #@param {type:"number"}

saved = []
for _ in range(n):
    generator = torch.manual_seed(seed)
    out = pipe(
        prompt=final_prompt,
        negative_prompt=negative or None,
        height=height, width=width,
        guidance_scale=cfg_scale,
        num_inference_steps=steps,
        generator=generator,
    )
    im = out.images[0]
    meta = {
        "mode": "text2img",
        "prompt": final_prompt,
        "negative": negative,
        "steps": steps,
        "cfg_scale": cfg_scale,
        "size": [height, width],
        "seed": seed,
        "timestamp": ts_now(),
    }
    p, _ = save_image_and_meta(im, prefix="text2img", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
    saved.append(p)

log_info(f"Text2Img saved: {saved}")


In [ ]:
# @title 📤 Upload image (опционально для Img2Img / ControlNet)
from google.colab import files
import os

# Можно заранее указать путь вручную:
src_path = ""  #@param {type:"string"}

uploaded = files.upload()  # откроет окно "Browse"

if uploaded:
    filename = list(uploaded.keys())[0]
    src_path = os.path.join("/content", filename)
    file_size = os.path.getsize(src_path) / 1024
    print(f"✅ Uploaded: {filename}")
    print(f"📂 Saved to: {src_path} ({file_size:.1f} KB)")
elif src_path:
    print(f"⚠️ No upload. Использую вручную заданный путь: {src_path}")
else:
    print("⚠️ No file uploaded and src_path пустой. Укажи источник изображения.")


In [ ]:
# @title 🖼 Img2Img
from scripts.pipelines import run_img2img

strength = 0.6   #@param {type:"number"}
seed     = 12345 #@param {type:"number"}
src_path = ""    #@param {type:"string"}

saved = run_img2img(
    user_prompt=user_prompt,
    style=base_style,
    tone=color_tone,
    negative=negative,
    src_path=src_path,
    CONFIG=CONFIG,
    DEFAULTS=DEFAULTS,
    strength=strength,
    seed=seed
)

print("✅ Img2Img completed.")
print("📂 Saved files:", saved if isinstance(saved, list) else [saved])


In [ ]:
# @title 📤 Upload image (опционально для ControlNet)
from google.colab import files
import os

uploaded = files.upload()  # откроет окно "Browse"

if uploaded:
    filename = list(uploaded.keys())[0]
    control_path = os.path.join("/content", filename)
    file_size = os.path.getsize(control_path) / 1024
    print(f"✅ Uploaded: {filename}")
    print(f"📂 Saved to: {control_path} ({file_size:.1f} KB)")
else:
    print("⚠️ No file uploaded. Используй control_path вручную в следующей ячейке.")



In [ ]:
# @title  🧭  ControlNet
from scripts.pipelines import run_controlnet

control_path = "" #@param {type:"string"}
low_thr      = 100   #@param {type:"number"}
high_thr     = 200   #@param {type:"number"}
seed         = 12345 #@param {type:"number"}

saved = run_controlnet(
    user_prompt=user_prompt,
    style=base_style,
    tone=color_tone,
    negative=negative,
    src_path=control_path,
    CONFIG=CONFIG,
    DEFAULTS=DEFAULTS,
    strength=1.0,
    seed=seed
)

print("✅ ControlNet completed.")
print("📂 Saved files:", saved if isinstance(saved, list) else [saved])
print(f"⚙️ Thresholds: low={low_thr}, high={high_thr}")


In [ ]:
# @title ⬆️ 10 Upscale x4
from PIL import Image

variant = CONFIG["MODEL_VARIANT"]
up_id   = VARIANT_MODELS[variant]["upscale"]

pipe_up = get_upscale_pipe(up_id, DEVICE, DTYPE)

in_path = ""   #@param {type:"string"}
seed    = 12345 #@param {type:"number"}

if not in_path:
    raise ValueError("Нужно указать путь к изображению для апскейла (in_path)")

img = Image.open(in_path).convert("RGB")
generator = torch.manual_seed(seed)

out = pipe_up(image=img, prompt=final_prompt, generator=generator)

im = out.images[0]
meta = {
    "mode": "upscale_x4",
    "prompt": final_prompt,
    "seed": seed,
    "timestamp": ts_now(),
}
p, _ = save_image_and_meta(im, prefix="upscale", meta=meta, output_dir=CONFIG["OUTPUT_DIR"])
log_info(f"Upscale saved: {p}")
